In [106]:
import numpy as np
from transitions import Machine

STEP_SIZE = 10
# Note: cooling is positive in GHz and heating is negative in GHz

NUM_OF_LASERS = 8   # number of lasers and/or MRMs
LASER_FREQ_SPACING = 400  # in GHz
LASER_FREQ_UNCERTAINTY = 50  # ± this number in GHz

MRM_FSR = 1800  # in GHz
MRM_FREQ_SPACING = 400  # in GHz
MRM_FREQ_UNCERTAINTY = 100  # in GHz
INITIAL_COOLDOWN = MRM_FSR // 2  # in GHz

# will track the resonances -1*FSR over, 0*FSR over, and 1*FSR over
WHICH_RESONANCES = list(range(-10, 11, 1))
BARREL_ROLL_SEQ = [0, 5, 1, 6, 2, 7, 3, 4]


class Lasers(object):

    def __init__(self,
                 variations=False,
                 corners=False,
                 reverse=False,
                 offset=0) -> None:
        self._delta = None

        # start by having all the laser lines 'bright'
        # will be replaced by the MRM number that captures that laser
        # Note: need to consider upstream vs. downstream
        self._brightness = ['bright'] * NUM_OF_LASERS

        # If reverse, the first laser line will have the highest frequency
        self._reverse = reverse

        # offset specifies the additive frequency offset from 0
        self._offset = offset

        self.populate_laser_lines(variations, corners)        

    def populate_laser_lines(self, variations, corners):
        # the first laser line is always at 0 GHz
        laser_lines = np.arange(
            NUM_OF_LASERS, dtype=int) * LASER_FREQ_SPACING

        if self._reverse:
            laser_lines = np.flip(laser_lines)
        
        laser_lines += self._offset    
    
        if variations:
            if corners:  # only evaluate the corners
                delta = np.random.randint(2, size=laser_lines.size)
                delta = 2*delta - 1  # convert to [-1, +1] random numbers
                delta *= LASER_FREQ_UNCERTAINTY
            else:
                delta = np.random.uniform(-1*LASER_FREQ_UNCERTAINTY,
                                          LASER_FREQ_UNCERTAINTY,
                                          laser_lines.size)
            # round to the nearest STEP_SIZE
            delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE
            delta = delta.astype(int)

            self._delta = delta
            laser_lines += delta

        self._laser_lines = laser_lines

    def capture_laser_line(self, which_laser, which_ring):
        assert self._brightness[which_laser] == 'bright', \
            f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be bright to be captured"
        # which_ring will modulate this laser
        self._brightness[which_laser] = which_ring

    @property
    def laser_lines(self):
        return self._laser_lines

    @property
    def brightness(self):
        return self._brightness

    def __str__(self) -> str:
        return f"Lines: {self.laser_lines}, Brightness: {self.brightness}"
    
    def __repr__(self) -> str:
        return self.__str__()


class RingFSM(object):

    states = ['idle', 'cold', 'search', 'lock']

    def __init__(self,
                 resonances,
                 lasers,
                 initial_loc=0,
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz
                 step_size=STEP_SIZE,
                 ) -> None:
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self.loc = initial_loc  # location of the resonance

        self.locked_laser_idx = None

        self._initial_loc = initial_loc
        self._previous_ring = previous_ring
        self._initial_resonances = resonances
        self._lasers = lasers

        self._cool_down_by = cool_down_by
        self._step_size = step_size
        self._found_all_lines = False

        self.machine = Machine(
            model=self, states=RingFSM.states, initial='idle')

        # 'start' starts the search algorithm
        self.machine.add_transition(trigger='start',
                                    source='idle',
                                    dest='cold',
                                    after='cool_step')

        # once the ring has cooled, start searching
        self.machine.add_transition(trigger='keep_searching',
                                    source='cold',
                                    dest='search',
                                    after='heat_step')

        # 'reset' puts the ring back to idle from any state
        self.machine.add_transition(trigger='reset',
                                    source='*',
                                    dest='idle',
                                    after='reset_resonances')

        # will 'keep_searching' until we find all the laser lines
        self.machine.add_transition(
            trigger='keep_searching',
            source='search',
            dest='search',
            after='heat_step')

        # lock ring once the laser lines have been found
        self.machine.add_transition(
            trigger='lock_to_line',
            source='search',
            dest='lock',
            after='lock_to_laser',
            conditions=['found_all_lines'])

    def reset_resonances(self):
        # reset everything
        self.loc = self._initial_loc
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self._found_all_lines = False

    def cool(self, amount):
        self.loc += amount

    def heat(self, amount):
        self.loc -= amount

    def cool_step(self):
        # cool the resonances by the cool down by amount
        self.cool(self._cool_down_by)

    def heat_step(self):
        # check if there's a laser signal to be found
        found = self._check_laser_signal()
        # found is tuple (bool, idx | None)
        # if found: (True, laser idx), and if not found: (False, None)
        if found[0]:
            self.laser_locs_seen.append(self.loc)
            self.laser_brightness_seen.append(
                self._lasers.brightness[found[1]])
            self.laser_indices_seen.append(found[1])

        # check if we have found all lasers
        self._check_exit_search_condition()

        # move the resonance location by specified step_size
        self.heat(self._step_size)

    def lock_to_laser(self):
        # jump to the laser signal we want to capture
        self.loc = self.laser_locs_seen[-2]
        # check and capture that laser signal
        self._check_laser_signal(capture=True)

    def _check_laser_signal(self, capture=False):
        # check if there's any laser signal at this location
        found_laser = False
        idx = None
        for r in self.resonances:
            if (r in self._lasers.laser_lines):
                # get the index
                idx = np.where(self._lasers.laser_lines == r)
                assert len(idx) == 1
                idx = idx[0][0]
                if capture:
                    if self._previous_ring is None:
                        this_ring = 0
                    else:
                        this_ring = self._previous_ring + 1
                    self._lasers.capture_laser_line(idx, this_ring)
                    self.locked_laser_idx = idx
                found_laser = True
        # if there's a laser there, found_laser will return that laser's index
        return (found_laser, idx)

    def _check_exit_search_condition(self):
        # if we have seen one dim and one bright, we have found all lines
        if self._previous_ring is None:  # this is the first ring
            if len(self.laser_locs_seen) >= 3:
                self._found_all_lines = True
        else:
            if len(self.laser_locs_seen) >= 3:
                if self.laser_brightness_seen[-3] != 'bright' \
                        and self.laser_brightness_seen[-2] == 'bright':
                    self._found_all_lines = True

    @property
    def found_all_lines(self):
        return self._found_all_lines

    @property
    def resonances(self):
        return self.loc + self._initial_resonances

In [107]:
from collections import deque
import copy


def populate_mrm_resonances(global_variations=False,
                            variations=False,
                            corners=False):

    mrm_lines = np.arange(NUM_OF_LASERS, dtype=np.int64) * MRM_FREQ_SPACING
    mrm_lines = np.vstack([mrm_lines] * len(WHICH_RESONANCES)).T

    fsr_shifts = np.array(WHICH_RESONANCES) * MRM_FSR
    mrm_lines += fsr_shifts

    if variations:  # assume we keep the FSR to be the same
        if corners:  # only evaluate the corners
            delta = np.random.randint(2, size=NUM_OF_LASERS)
            delta = 2*delta - 1  # convert to [-1, +1] random numbers
            delta *= MRM_FREQ_UNCERTAINTY
        else:
            delta = np.random.uniform(-1*MRM_FREQ_UNCERTAINTY,
                                      MRM_FREQ_UNCERTAINTY,
                                      NUM_OF_LASERS)
        # round to the nearest STEP_SIZE
        delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE

        delta = delta.astype(int)
        delta = delta[:, np.newaxis]
        mrm_lines += delta

    if global_variations:  # the global variation can be as much as ±1 FSR
        global_delta = np.random.uniform(-1*MRM_FSR, MRM_FSR)
        global_delta = np.round(global_delta/STEP_SIZE, decimals=0) * STEP_SIZE
        global_delta = global_delta.astype(int)
        mrm_lines += global_delta

    return mrm_lines


def check_barrel_roll(list1, list2):
    # check for barrel roll in sequence or reverse sequence
    assert len(list1) == len(list2)

    def check_equivalent(list1, list2):
        d1 = deque(list1)
        d2 = deque(list2)

        equivalent = d1 == d2

        for _ in range(len(list1)):
            d1.rotate(1)
            if d1 == d2:
                equivalent = True

        return equivalent
    
    list1_ = copy.deepcopy(list1)
    equivalent = check_equivalent(list1_, list2)
    list1_.reverse()
    equivalent = equivalent or check_equivalent(list1_, list2)
    return equivalent

In [108]:
# Try out the algorithm with a single try
lasers = Lasers(variations=True, corners=True, reverse=True, offset=-8*LASER_FREQ_SPACING)
all_ring_resonances = populate_mrm_resonances(variations=True, corners=True)

all_rings = []

for idx in range(NUM_OF_LASERS):
    if idx == 0:
        mrm = RingFSM(resonances=all_ring_resonances[idx], lasers=lasers)
    else:
        mrm = RingFSM(resonances=all_ring_resonances[idx], lasers=lasers, previous_ring=idx-1)
    mrm.start()

    while not mrm.found_all_lines:
        mrm.keep_searching()

    mrm.lock_to_line()
    all_rings.append(mrm)

    print(f"Locked to laser # {mrm.locked_laser_idx}")
    print(lasers.brightness)

lasers_locked_to = [int(m.locked_laser_idx) for m in all_rings]

# check for barrel roll in sequence or reverse sequence
print("Barrel roll check?", check_barrel_roll(lasers_locked_to, BARREL_ROLL_SEQ))

amt_heat_applied = [m.loc for m in all_rings]
print(f"Amount of heat applied: {amt_heat_applied}")

Locked to laser # 6
['bright', 'bright', 'bright', 'bright', 'bright', 'bright', 0, 'bright']
Locked to laser # 2
['bright', 'bright', 1, 'bright', 'bright', 'bright', 0, 'bright']
Locked to laser # 7
['bright', 'bright', 1, 'bright', 'bright', 'bright', 0, 2]
Locked to laser # 3
['bright', 'bright', 1, 3, 'bright', 'bright', 0, 2]
Locked to laser # 4
['bright', 'bright', 1, 3, 4, 'bright', 0, 2]
Locked to laser # 0
[5, 'bright', 1, 3, 4, 'bright', 0, 2]
Locked to laser # 5
[5, 'bright', 1, 3, 4, 6, 0, 2]
Locked to laser # 1
[5, 7, 1, 3, 4, 6, 0, 2]
Barrel roll check? True
Amount of heat applied: [750, 250, -350, -950, 50, -650, 550, -150]


In [115]:
# Run Monte Carlo and get distribution

import pandas as pd
import copy

eval_corners = False

# cycling_freqs = [-k * MRM_FSR // 9 for k in range(9)]  # change in frequency due to cycling
cycling_freqs = [0]

num_trials = 50
trial_results = []

for _ in range(num_trials):
    lasers = Lasers(variations=True, corners=eval_corners, reverse=True, offset=-8*LASER_FREQ_SPACING)
    all_ring_resonances = populate_mrm_resonances(global_variations=True,
                                                  variations=True,
                                                  corners=eval_corners)

    for ctry, cfreq in enumerate(cycling_freqs):

        lasers_ = copy.deepcopy(lasers)
        all_rings = []

        for idx in range(NUM_OF_LASERS):
            if idx == 0:  # the first ring
                mrm = RingFSM(resonances=all_ring_resonances[idx],
                              lasers=lasers_,
                              initial_loc=cfreq)
            else:
                mrm = RingFSM(resonances=all_ring_resonances[idx],
                              lasers=lasers_,
                              initial_loc=cfreq,
                              previous_ring=idx-1)
            # Run the search algorithm for each MRM
            mrm.start()

            while not mrm.found_all_lines:
                mrm.keep_searching()

            mrm.lock_to_line()
            all_rings.append(mrm)

        laser_sequence = [int(m.locked_laser_idx) for m in all_rings]
        amt_heat_applied = [m.loc for m in all_rings]
        colder = [n for n in amt_heat_applied if n >= 0]
        hotter = [n for n in amt_heat_applied if n <= 0]

        max_cool = max(colder) if len(colder) > 0 else 0
        max_heat = min(hotter) if len(hotter) > 0 else 0
        
        max_diff = max_cool - max_heat

        # if True, cycle; else, success and no need for cycling
        cycling_condition = abs(max_heat) > MRM_FSR // 2
        if not cycling_condition:
            break

    result = {'laser_sequence': laser_sequence,
              'barrel_roll': check_barrel_roll(laser_sequence, BARREL_ROLL_SEQ), # check if barrel roll equivalent
              'max_cool': max_cool,
              'max_heat': max_heat,
              'max_diff': max_diff,
              'cycling_trial': ctry,
              'successful_lock': not cycling_condition  # check if we were able to lock with < 1/2 * FSR of heat
              }
    
    ring_numbers = list(range(NUM_OF_LASERS))
    heat_result = dict(zip(ring_numbers, amt_heat_applied))  # (-) value is heat and (+) value is cool
    result.update(heat_result)

    trial_results.append(result)

trial_results = pd.DataFrame(trial_results)



In [116]:
# Now plot the results of the Monte Carlo runs
import plotly.express as px

# Heat and cool amount
fig = px.histogram(np.array(trial_results[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = 100)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-MRM_FSR, MRM_FSR+tick_size, tick_size))
    )
)
fig.show()

# Plot the maximum differences
fig = px.histogram(trial_results, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = 100)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*MRM_FSR+tick_size, tick_size))
    )
)
fig.show()

In [117]:
# check if all the results in barrel-roll sequence

print("All in barrel-roll sequence? ", trial_results['barrel_roll'].all())


All in barrel-roll sequence?  True


In [118]:
# display all the trial results in a markdown table
from IPython.display import display, Markdown
display(Markdown(trial_results.to_markdown()))

|    | laser_sequence           | barrel_roll   |   max_cool |   max_heat |   max_diff |   cycling_trial | successful_lock   |   0 |    1 |     2 |     3 |    4 |    5 |     6 |    7 |
|---:|:-------------------------|:--------------|-----------:|-----------:|-----------:|----------------:|:------------------|----:|-----:|------:|------:|-----:|-----:|------:|-----:|
|  0 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        600 |       -910 |       1510 |               0 | False             | 500 |  -10 |  -560 |   600 | -240 | -910 |   490 | -310 |
|  1 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        610 |       -750 |       1360 |               0 | True              | 590 |  130 |  -490 |   610 |  -60 | -750 |   480 |  -80 |
|  2 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        620 |       -680 |       1300 |               0 | True              | 560 |    0 |  -680 |   620 |   30 | -550 |   350 | -190 |
|  3 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        570 |       -780 |       1350 |               0 | True              | 570 |  -60 |  -540 |   560 |   10 | -780 |   430 | -130 |
|  4 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        590 |       -620 |       1210 |               0 | True              | 560 |   70 |  -550 |   590 | -130 | -620 |   480 |  -50 |
|  5 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        650 |       -730 |       1380 |               0 | True              | 650 |  -10 |  -610 |   420 | -210 | -730 |   310 | -180 |
|  6 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        310 |       -910 |       1220 |               0 | False             | 250 | -370 |  -860 |   180 | -350 | -910 |   310 | -290 |
|  7 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        700 |      -1090 |       1790 |               0 | False             | 560 |   20 |  -650 |   700 |   40 | -570 | -1090 | -190 |
|  8 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        700 |      -1250 |       1950 |               0 | False             | 700 |  190 |  -520 | -1250 | -130 | -730 |   550 |  -80 |
|  9 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        530 |       -920 |       1450 |               0 | False             | 490 |  -40 |  -710 |   530 | -240 | -920 |   260 | -300 |
| 10 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        530 |      -1030 |       1560 |               0 | False             | 530 | -180 | -1030 |   160 | -250 | -970 |   190 | -510 |
| 11 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        640 |      -1140 |       1780 |               0 | False             | 640 |  -10 |  -520 | -1140 |  -50 | -600 |   410 |  -30 |
| 12 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        640 |       -740 |       1380 |               0 | True              | 600 |   30 |  -510 |   640 | -140 | -740 |   510 | -150 |
| 13 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        490 |       -910 |       1400 |               0 | False             | 490 | -140 |  -600 |   430 | -260 | -910 |   300 | -350 |
| 14 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        700 |       -650 |       1350 |               0 | True              | 460 |  -20 |  -580 |   570 |    0 | -650 |   700 | -320 |
| 15 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        640 |       -720 |       1360 |               0 | True              | 640 |  130 |  -710 |   460 | -170 | -720 |   540 |  -10 |
| 16 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        560 |       -900 |       1460 |               0 | True              | 560 |   20 |  -510 |   550 | -210 | -900 |   360 | -280 |
| 17 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        540 |       -950 |       1490 |               0 | False             | 540 |  -30 |  -880 |   340 | -270 | -950 |   330 | -330 |
| 18 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        490 |       -830 |       1320 |               0 | True              | 490 |  -40 |  -780 |   320 | -210 | -830 |   420 | -380 |
| 19 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        650 |       -900 |       1550 |               0 | True              | 650 |  230 |  -380 |  -900 |   40 | -640 |   570 |  -90 |
| 20 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        570 |       -840 |       1410 |               0 | True              | 570 | -160 |  -730 |   570 |  -10 | -840 |   490 | -230 |
| 21 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        710 |       -670 |       1380 |               0 | True              | 590 |  -20 |  -530 |   650 |   40 | -670 |   710 | -200 |
| 22 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        570 |       -850 |       1420 |               0 | True              | 570 |  -10 |  -620 |   550 |  -80 | -850 |   280 | -280 |
| 23 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        350 |       -980 |       1330 |               0 | False             | 350 | -310 |  -930 |   210 | -340 | -980 |   210 | -270 |
| 24 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        640 |       -820 |       1460 |               0 | True              | 640 | -370 |  -820 |   450 | -220 | -800 |   260 | -250 |
| 25 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        600 |       -910 |       1510 |               0 | False             | 600 |  -60 |  -540 |   600 |   10 | -910 |   500 | -210 |
| 26 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        700 |       -820 |       1520 |               0 | True              | 700 | -120 |  -820 |   590 |  -20 | -800 |   460 | -180 |
| 27 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        590 |       -710 |       1300 |               0 | True              | 590 |  -80 |  -690 |   490 | -120 | -710 |   530 | -180 |
| 28 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        480 |       -810 |       1290 |               0 | True              | 280 | -270 |  -750 |   440 |  -80 | -810 |   480 | -210 |
| 29 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        510 |       -900 |       1410 |               0 | True              | 510 | -120 |  -760 |   420 | -270 | -900 |   330 | -110 |
| 30 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        630 |       -620 |       1250 |               0 | True              | 630 | -150 |  -620 |   480 | -120 | -590 |   470 | -100 |
| 31 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        560 |       -890 |       1450 |               0 | True              | 560 |  -50 |  -890 |   280 | -410 | -850 |   150 | -430 |
| 32 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        670 |       -780 |       1450 |               0 | True              | 670 |  160 |  -500 |   670 | -110 | -780 |   470 |  -80 |
| 33 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        670 |       -630 |       1300 |               0 | True              | 670 |   40 |  -550 |   500 |  -70 | -630 |   530 |  -20 |
| 34 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        540 |       -780 |       1320 |               0 | True              | 460 |  -90 |  -720 |   540 | -130 | -780 |   540 | -300 |
| 35 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        460 |       -980 |       1440 |               0 | False             | 350 | -190 |  -810 |   410 | -230 | -980 |   460 | -560 |
| 36 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        660 |       -760 |       1420 |               0 | True              | 460 | -140 |  -760 |   660 |   20 | -690 |   490 | -230 |
| 37 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        570 |       -750 |       1320 |               0 | True              | 570 |   20 |  -750 |   470 | -100 | -680 |   500 | -160 |
| 38 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        520 |       -700 |       1220 |               0 | True              | 520 | -210 |  -700 |   510 | -210 | -690 |   500 | -170 |
| 39 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        650 |       -740 |       1390 |               0 | True              | 650 | -140 |  -740 |   480 |  -30 | -590 |   450 | -110 |
| 40 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        590 |       -670 |       1260 |               0 | True              | 590 |  -60 |  -660 |   570 | -150 | -670 |   290 | -220 |
| 41 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        710 |       -690 |       1400 |               0 | True              | 610 |  -20 |  -570 |   710 |  -70 | -690 |   640 |  -90 |
| 42 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        630 |      -1310 |       1940 |               0 | False             | 560 |  -50 |  -650 |   630 |  170 | -500 | -1310 | -130 |
| 43 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        580 |       -880 |       1460 |               0 | True              | 520 | -110 |  -670 |   580 | -250 | -880 |   400 | -220 |
| 44 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        680 |       -620 |       1300 |               0 | True              | 680 |  120 |  -540 |   630 |  -20 | -620 |   390 | -190 |
| 45 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        450 |       -970 |       1420 |               0 | False             | 330 | -410 |  -970 |   310 | -240 | -840 |   450 | -370 |
| 46 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        510 |       -810 |       1320 |               0 | True              | 510 |  -80 |  -810 |   510 |   40 | -710 |   510 | -300 |
| 47 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        650 |      -1110 |       1760 |               0 | False             | 650 |  -20 |  -500 | -1110 |  -80 | -660 |   410 |  -60 |
| 48 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        530 |       -780 |       1310 |               0 | True              | 510 | -170 |  -780 |   530 | -120 | -690 |   200 | -250 |
| 49 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        570 |       -720 |       1290 |               0 | True              | 570 |  -20 |  -720 |   570 |    0 | -630 |   510 | -220 |